In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model

Using TensorFlow backend.


In [2]:
def awgn(x_input, reqSNR):
    sig_power = 1
    
    noise_power = np.sqrt(sig_power / (2*reqSNR))
    noise = noise_power * np.random.normal(size = x_input.shape)
    
    return x_input + noise

In [3]:
P = np.array([
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
], dtype='int')

In [4]:
H = np.concatenate((np.eye(11, dtype="int"), P.T), axis=1)
G= np.concatenate((P, np.eye(12, dtype="int")), axis=1)

In [5]:
model = load_model('golay_ex.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [6]:
for i in range(100):
    train_size = 10**6
    m = np.random.randint(2, size = (train_size, 12))
    train = np.dot(m,G) % 2
    train = np.where(train > 0, -1, 1)

    test_size = 10**3
    m = np.random.randint(2, size = (test_size, 12))
    test = np.dot(m,G) % 2
    test = np.where(test > 0, -1, 1)
    
    Eb_No = 10**(3/10)
    Es_No = Eb_No * (12/23)

    train_noise = awgn(train, Es_No)
    test_noise = awgn(test, Es_No)
    
    model.fit(train_noise, train,
          epochs=100,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_noise, test),verbose=0)
    
    if i%10 == 0:
        loss = model.evaluate(test_noise, test)
        print(loss)
        
        model.save('golay_ex.h5')

1000/1000 [==============================] - 0s 57us/step
[0.11967508560419082, 0.13500000536441803]
1000/1000 [==============================] - 0s 55us/step
[0.08210522937774659, 0.1120000034570694]
1000/1000 [==============================] - 0s 66us/step
[0.07469530373811722, 0.09399999678134918]
1000/1000 [==============================] - 0s 72us/step
[0.07977388854324817, 0.07000000029802322]
1000/1000 [==============================] - 0s 68us/step
[0.08007070648670196, 0.07199999690055847]
1000/1000 [==============================] - 0s 71us/step
[0.08457481467723846, 0.07900000363588333]
1000/1000 [==============================] - 0s 74us/step
[0.08122447723150253, 0.0949999988079071]
1000/1000 [==============================] - 0s 68us/step
[0.08771157410740853, 0.052000001072883606]
1000/1000 [==============================] - 0s 73us/step
[0.08571785950660706, 0.08900000154972076]
1000/1000 [==============================] - 0s 76us/step
[0.0780793662071228, 0.07400000095

In [7]:
model.save('golay_ex.h5')